In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

In [2]:
# Loading the datasets
data_train = np.load(r"Dataset/Shoes - Female - Train.npz")
data_val = np.load(r"Dataset/Shoes - Female - Validation.npz")
data_test = np.load(r"Dataset/Shoes - Female - Test.npz")

In [3]:
# Extracting the arrays from the imported data
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_val['images']
labels_val = data_val['labels']

images_test = data_test['images']
labels_test = data_test['labels']

In [4]:
# Scaling the pixel values of all images
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

In [5]:
# Defining the hyperparameters we would tune, and their values to be tested
HP_FILTER_SIZE_1 = hp.HParam('filter_size_1', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filters_number', hp.Discrete([64,96,128]))
HP_FILTER_SIZE_2 = hp.HParam('filter_size_2', hp.Discrete([3,5]))
HP_DENSE_SIZE = hp.HParam('dense_size', hp.Discrete([256]))#,512,1024]))

METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer(r'Logs/Model_Female/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE_1, HP_FILTER_NUM, HP_FILTER_SIZE_2, HP_DENSE_SIZE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [6]:
EPOCHS = 20
BATCH_SIZE = 64

In [7]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE_1], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE_2], activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_DENSE_SIZE], activation='relu'),
        tf.keras.layers.Dense(6)
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    
    # Defining the logging directory
    log_dir = "Logs/Model_Female/fit/" + "run-{}".format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure  
    
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
    
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Boots', 'Ballerina shoes', 'Trainers/Sneakers', 'High heels', 'Sandals/Flip flops/Slippers', 'Others'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val,labels_val),
        verbose = 2
    )
    
    
    # Evaluating the model's performance on the validation set
    _, accuracy = model.evaluate(images_val,labels_val)
    
    # Saving the current model for future reference
    model.save(r"saved_models/Model_Female/Run-{}".format(session_num))
    
    return accuracy

In [8]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [9]:
session_num = 1

for filter_size_1 in HP_FILTER_SIZE_1.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        for filter_size_2 in HP_FILTER_SIZE_2.domain.values:
            for dense_size in HP_DENSE_SIZE.domain.values:
            
                hparams = {
                    HP_FILTER_SIZE_1: filter_size_1,
                    HP_FILTER_NUM: filter_num,
                    HP_FILTER_SIZE_2: filter_size_2,
                    HP_DENSE_SIZE: dense_size
                }

                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('Logs/Model_Female/hparam_tuning/' + run_name, hparams, session_num)

                session_num += 1

--- Starting trial: run-1
{'filter_size_1': 3, 'filters_number': 64, 'filter_size_2': 3, 'dense_size': 256}
Epoch 1/20
46/46 - 60s - loss: 1.3276 - accuracy: 0.5277 - val_loss: 0.6697 - val_accuracy: 0.7639
Epoch 2/20
46/46 - 52s - loss: 0.5513 - accuracy: 0.8139 - val_loss: 0.4640 - val_accuracy: 0.8500
Epoch 3/20
46/46 - 54s - loss: 0.4408 - accuracy: 0.8472 - val_loss: 0.4991 - val_accuracy: 0.8500
Epoch 4/20
46/46 - 56s - loss: 0.3634 - accuracy: 0.8770 - val_loss: 0.3957 - val_accuracy: 0.8611
Epoch 5/20
46/46 - 114s - loss: 0.2723 - accuracy: 0.9106 - val_loss: 0.4755 - val_accuracy: 0.8472
Epoch 6/20
46/46 - 105s - loss: 0.2467 - accuracy: 0.9189 - val_loss: 0.4897 - val_accuracy: 0.8528
12/12 [==============================] - 4s 286ms/step - loss: 0.3957 - accuracy: 0.8611
INFO:tensorflow:Assets written to: saved_models/Model_Female/Run-1/assets
--- Starting trial: run-2
{'filter_size_1': 3, 'filters_number': 64, 'filter_size_2': 5, 'dense_size': 256}
Epoch 1/20
46/46 - 180s -

12/12 [==============================] - 5s 394ms/step - loss: 0.4567 - accuracy: 0.8667
INFO:tensorflow:Assets written to: saved_models/Model_Female/Run-9/assets
--- Starting trial: run-10
{'filter_size_1': 5, 'filters_number': 96, 'filter_size_2': 5, 'dense_size': 256}
Epoch 1/20
46/46 - 256s - loss: 1.2859 - accuracy: 0.5596 - val_loss: 0.6587 - val_accuracy: 0.7833
Epoch 2/20
46/46 - 2868s - loss: 0.7076 - accuracy: 0.7751 - val_loss: 0.6640 - val_accuracy: 0.8028
Epoch 3/20
46/46 - 356s - loss: 0.5107 - accuracy: 0.8222 - val_loss: 0.9028 - val_accuracy: 0.6972
12/12 [==============================] - 12s 985ms/step - loss: 0.6587 - accuracy: 0.7833
INFO:tensorflow:Assets written to: saved_models/Model_Female/Run-10/assets
--- Starting trial: run-11
{'filter_size_1': 5, 'filters_number': 128, 'filter_size_2': 3, 'dense_size': 256}
Epoch 1/20
46/46 - 338s - loss: 1.9291 - accuracy: 0.4238 - val_loss: 0.7489 - val_accuracy: 0.7333
Epoch 2/20
46/46 - 276s - loss: 0.6933 - accuracy: 0

--- Starting trial: run-18
{'filter_size_1': 7, 'filters_number': 128, 'filter_size_2': 5, 'dense_size': 256}
Epoch 1/20
46/46 - 697s - loss: 1.9916 - accuracy: 0.3732 - val_loss: 1.1543 - val_accuracy: 0.5139
Epoch 2/20
46/46 - 556s - loss: 0.8081 - accuracy: 0.7225 - val_loss: 0.9166 - val_accuracy: 0.6306
Epoch 3/20
46/46 - 622s - loss: 0.6073 - accuracy: 0.7814 - val_loss: 0.5396 - val_accuracy: 0.8306
Epoch 4/20
46/46 - 566s - loss: 0.4821 - accuracy: 0.8365 - val_loss: 0.4810 - val_accuracy: 0.8722
Epoch 5/20
46/46 - 555s - loss: 0.3719 - accuracy: 0.8687 - val_loss: 0.4289 - val_accuracy: 0.8694
Epoch 6/20
46/46 - 500s - loss: 0.3141 - accuracy: 0.8957 - val_loss: 0.4867 - val_accuracy: 0.8639
Epoch 7/20
46/46 - 3093s - loss: 0.2874 - accuracy: 0.9044 - val_loss: 0.4532 - val_accuracy: 0.8694
12/12 [==============================] - 12s 992ms/step - loss: 0.4289 - accuracy: 0.8694
INFO:tensorflow:Assets written to: saved_models/Model_Female/Run-18/assets


In [10]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"saved_models/Model_Female/Run-1")

In [11]:
test_loss, test_accuracy = model.evaluate(images_test,labels_test)

12/12 [==============================] - 4s 242ms/step - loss: 0.5377 - accuracy: 0.8194


In [12]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.5377. Test accuracy: 81.94%


In [ ]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model_Female/hparam_tuning"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model_Female/fit"